In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pandas as pd
import bs4
from selenium import webdriver
import re

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [37]:
driver_path = 'D:/Downloads/chromedriver_win32/chromedriver.exe'
#opts = webdriver.ChromeOptions()
#opts.headless = True
driver = webdriver.Chrome()


In [38]:
def get_info(film_tag):
    
    hrefMovie = film_tag.find('div',class_ = 'lister-item-image')
    hrefMovie = hrefMovie.find('a')['href']
    href_link = 'https://www.imdb.com/' + hrefMovie
    driver.get(href_link)
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div[data-testid='storyline-plot-keywords']")))
    html_text = driver.page_source
    href_soup = BeautifulSoup(html_text)
    
    keywords_tag = href_soup.find_all('div', class_="ipc-chip-list__scroller")
    keywords = []
    if keywords_tag[1]:
        for tag in keywords_tag[1].contents:
            keywords.append(tag.get_text())
        keywords.pop(-1)
    
    tagline_tag = href_soup.select('li[data-testid="storyline-taglines"] label')
    tagline = ''
    if tagline_tag:
        tagline = [i.text for i in tagline_tag][0]
        
    metascore_tag = href_soup.find('span', class_ = 'score-meta')
    metascore = ''
    if metascore_tag:
        metascore = metascore_tag.get_text()
    else:
        metascore = np.nan
    
    details = href_soup.select('div[data-testid="title-details-section"] a[class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
    release_date = ''
    countries = []
    language = []
    locations = []
    company = []
    if details:
        for d in details:
            if 'releaseinfo' in d['href']:
                release_date = d.text
            elif 'country_of_origin' in d['href']:
                countries.append(d.text)
            elif 'primary_language' in d['href']:
                language.append(d.text)
            elif 'locations' in d['href']:
                locations.append(d.text)
            elif 'company' in d['href']:
                company.append(d.text)
    
    scraped_list = film_tag.find( 'div', class_ = 'lister-item-content')
    
    name_tag = film_tag.find('h3')
    name = name_tag.get_text().split('\n')[2]
    
    muted_text = film_tag.find_all( 'p', class_ = 'text-muted')
    overview = muted_text[1].get_text().strip()
    
    cer = film_tag.find('span', class_ = 'certificate')
    if cer:
        certificate = cer.get_text()
    else:
        certificate = ''
    
    runtime_tag = film_tag.find('span', class_ = 'runtime')
    if runtime_tag:
        runtime = float(runtime_tag.get_text().split(' ')[0])
    else:
        runtime = np.nan
        
    genre_tag = film_tag.find('span', class_ = 'genre')
    if genre_tag:
        genre = genre_tag.get_text().strip().split(', ')
    else: 
        genre = ''
    
    imdb_rate_tag = film_tag.find('strong')
    if imdb_rate_tag:
        imdb_rate = float(imdb_rate_tag.get_text())
    else:
        imdb_rate = np.nan
#     metascore_tag = film_tag.find('span', class_ = 'metascore favorable')
#     if metascore_tag :
#         metascore = float(metascore_tag.get_text().strip())
#     else:
#         metascore =np.nan
    star_tag = film_tag.find('p', class_ = "").get_text().strip()
    star_tag = star_tag.split('|')
    if len(star_tag) > 1:
        director_tag =  star_tag[0].split(':\n')
        star_tag = star_tag[1].split(':\n')
        director = director_tag[1].strip().split(', \n')
        stars = star_tag[1].strip().split(', \n')
    else:
        temp = star_tag[0].strip().split(':\n')
        if temp[0] == 'Director':
            director = temp[1]
            stars = ''
        else:
            stars = temp[1]
            director = ''
        
    vote_tag = film_tag.find('p', class_ = 'sort-num_votes-visible').get_text()
    vote_tag = vote_tag.split('\n|')
    nvote = vote_tag[0].split(':\n')[1].replace(',', '')
    nvote = int(nvote)
    if len(vote_tag) > 1:
        gross = float(vote_tag[1].strip().split(':\n')[1].strip('$M'))
    else:
        gross = np.nan
    
    result = [name, overview,tagline,certificate, runtime, genre,keywords, imdb_rate, metascore, director, stars, nvote, gross,
              release_date, countries,language, locations, company]
    return result

In [39]:
film_list = []
# search feature film có nhiều lượt vote từ 2010 - 2021, 
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2021-12-31&sort=num_votes,desc&count=250"
while len(film_list) < 1000:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    scraped_list = soup.find_all( 'div', class_ = 'lister-item mode-advanced')
    for film in scraped_list:
        film_list.append(get_info(film))
    next_tag = soup.find('a', class_ = 'lister-page-next next-page')
    url = 'https://www.imdb.com/' + next_tag.get('href')
key = ['name', 'overview','tagline','certificate', 'runtime', 'genre','keywords', 'imdb_rate', 'metascore', 'director', 'stars', 'nvote', 'gross',
              'release_date', 'countries','language', 'locations', 'company']
df = pd.DataFrame(film_list,columns = key)

In [45]:
driver.quit()

In [40]:
df

,name,overview,tagline,certificate,runtime,genre,keywords,imdb_rate,metascore,director,stars,nvote,gross,release_date,countries,language,locations,company
0,Kẻ Đánh Cắp Giấc Mơ,A thief who steals corporate secrets through t...,Your mind is the scene of the crime,PG-13,148.0,"[Action, Adventure, Sci-Fi]","[dream, ambiguous ending, subconscious, mindbe...",8.8,74.0,[Christopher Nolan],"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",2342938,292.58,"July 16, 2010 (United States)","[United States, United Kingdom]","[English, Japanese, French]","[Fortress Mountain, Kananaskis Country, Albert...","[Warner Bros., Legendary Entertainment, Syncopy]"
1,Hố Đen Tử Thần,A team of explorers travel through a wormhole ...,Mankind was born on Earth. It was never meant ...,C13,169.0,"[Adventure, Drama, Sci-Fi]","[astronaut, saving the world, space travel, wo...",8.6,74.0,[Christopher Nolan],"[Matthew McConaughey, Anne Hathaway, Jessica C...",1819025,188.02,"November 7, 2014 (United States)","[United States, United Kingdom, Canada]",[English],[Iceland],"[Paramount Pictures, Warner Bros., Legendary E..."
2,Kỵ Sĩ Bóng Đêm Trỗi Dậy,Eight years after the Joker's reign of anarchy...,The Legend Ends,PG-13,164.0,"[Action, Drama]","[dc comics, batman character, bruce wayne char...",8.4,78.0,[Christopher Nolan],"[Christian Bale, Tom Hardy, Anne Hathaway, Gar...",1700671,448.14,"July 20, 2012 (United States)","[United States, United Kingdom]","[English, Arabic]","[Mehrangarh Fort, Jodhpur, Rajasthan, India]","[Warner Bros., Legendary Entertainment, DC Ent..."
3,Hành Trình Django,"With the help of a German bounty-hunter, a fre...","Life, liberty and the pursuit of vengeance.",R,165.0,"[Drama, Western]","[racial vengeance, racial violence, slavery, o...",8.4,81.0,[Quentin Tarantino],"[Jamie Foxx, Christoph Waltz, Leonardo DiCapri...",1548179,162.81,"December 25, 2012 (United States)",[United States],"[English, German, French, Italian]","[Evergreen Plantation, 4677 Highway 18, Edgard...","[The Weinstein Company, Columbia Pictures]"
4,Sói Già Phố Wall,"Based on the true story of Jordan Belfort, fro...",Earn. Spend. Party.,R,180.0,"[Biography, Comedy, Crime]","[based on true story, stockbroker, female nudi...",8.2,75.0,[Martin Scorsese],"[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",1403561,116.90,"December 25, 2013 (United States)",[United States],"[English, French]","[Portofino, Genoa, Liguria, Italy]","[Red Granite Pictures, Appian Way, Sikelia Pro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,To Rome with Love,The lives of some visitors and residents of Ro...,,R,112.0,"[Comedy, Music, Romance]","[leftist radical, written and directed by cast...",6.3,NaN,[Woody Allen],"[Woody Allen, Penélope Cruz, Jesse Eisenberg, ...",88527,16.69,"July 6, 2012 (United States)","[United States, Italy, Spain]","[English, Italian]","[Piazza Venezia, Rome, Lazio, Italy]","[Medusa Film, Gravier Productions, Perdido Pro..."
996,Niệm Khúc Cuối,A rebellious girl is sent to a Southern beach ...,Do you ever really forget your first heartbreak?,PG,108.0,"[Drama, Music, Romance]","[father daughter relationship, love, beach, pi...",6.0,NaN,[Julie Anne Robinson],"[Miley Cyrus, Liam Hemsworth, Greg Kinnear, Bo...",88476,62.95,"March 31, 2010 (United States)",[United States],[English],"[Tybee Island, Georgia, USA]","[Touchstone Pictures, Offspring Entertainment,..."
997,Cậu Ấm Quái Chiêu,A comedy centered on an idealist who barges in...,Everybody has one.,R,90.0,"[Comedy, Drama]","[dog, golden retriever, lesbian, parole office...",6.4,NaN,[Jesse Peretz],"[Paul Rudd, Elizabeth Banks, Zooey Deschanel, ...",88475,24.81,"August 26, 2011 (United States)",[United States],[English],"[Cold Spring, New York, USA]","[Big Beach Films, Likely Story, The Weinstein ..."
998,Rắc Rối Tuổi Teen,A 20-something supervising staff member of a r...,Support them. Take care of them. But don't bec...,R,96.0,[Drama],"[teenager, self harm, strong female character,...",7

In [44]:
df.to_csv('movies_data.csv')